In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

# Create 5 multi-server queues with FCFS scheduling
node = np.empty(5, dtype=object)
node[0] = Queue(model, 'Queue1', SchedStrategy.FCFS)
node[0].set_number_of_servers(1)
node[1] = Queue(model, 'Queue2', SchedStrategy.FCFS)
node[1].set_number_of_servers(2)
node[2] = Queue(model, 'Queue3', SchedStrategy.FCFS)
node[2].set_number_of_servers(3)
node[3] = Queue(model, 'Queue4', SchedStrategy.FCFS)  # only closed classes
node[3].set_number_of_servers(4)
node[4] = Queue(model, 'Queue5', SchedStrategy.FCFS)  # only open classes
node[4].set_number_of_servers(5)

source = Source(model, 'Source')
sink = Sink(model, 'Sink')

# Create job classes
jobclass = np.empty(2, dtype=object)
jobclass[0] = ClosedClass(model, 'ClosedClass', 3, node[0], 0)
jobclass[1] = OpenClass(model, 'OpenClass', 0)

# Set service rates
for i in range(5):
    node[i].set_service(jobclass[0], Exp(i+1))
    node[i].set_service(jobclass[1], Exp(np.sqrt(i+1)))

# Set arrival rate for open class
source.set_arrival(jobclass[1], Exp(0.3))

In [3]:
# Setup routing matrix
M = model.get_number_of_stations()
K = model.get_number_of_classes()

P = model.init_routing_matrix()

# Closed class routing: Queue1 -> Queue2 -> Queue3 -> Queue4 -> Queue1 (circular)
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)
P.set(jobclass[0], jobclass[0], node[1], node[2], 1.0)
P.set(jobclass[0], jobclass[0], node[2], node[3], 1.0)
P.set(jobclass[0], jobclass[0], node[3], node[0], 1.0)

# Open class routing: Source -> Queue1 -> Queue2 -> Queue3 -> Queue5 -> Sink
P.set(jobclass[1], jobclass[1], source, node[0], 1.0)
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)
P.set(jobclass[1], jobclass[1], node[1], node[2], 1.0)
P.set(jobclass[1], jobclass[1], node[2], node[4], 1.0)
P.set(jobclass[1], jobclass[1], node[4], sink, 1.0)

model.link(P)

In [4]:
print('This example shows the execution of the solver on a 2-class mixed model with 5 multi-server nodes.')

# Create solvers
solver = np.array([], dtype=object)
solver = np.append(solver, CTMC(model, cutoff=3))  # CTMC is infinite on this model
solver = np.append(solver, JMT(model, samples=100000, seed=23000))
solver = np.append(solver, SSA(model, cutoff=3, seed=23000))
solver = np.append(solver, MVA(model))
# Note: MAM does not support mixed models with both open and closed classes
solver = np.append(solver, DES(model, samples=100000, seed=23000))

# Solve and display results
AvgTable = np.empty(len(solver), dtype=object)
for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].get_name()}')
    AvgTable[s] = solver[s].avg_table()
    print(AvgTable[s])

This example shows the execution of the solver on a 2-class mixed model with 5 multi-server nodes.

SOLVER: CTMC
CTMC solver using state space cutoff = 3 for open/mixed model.


/home/gcasale/Dropbox/code/line-dev.git/python/line_solver/api/solvers/ctmc/handler.py:834: UserWarning: State space truncation may cause inaccurate results. Consider varying cutoff to assess sensitivity.
  warnings.warn(


Station    JobClass       QLen       Util  RespT     ResidT       ArvR       Tput
 Queue1 ClosedClass 1.0258e+00 4.3030e-01 2.3840 1.3678e-01 4.3030e-01 4.3030e-01
 Queue1   OpenClass 7.2053e-01 2.8229e-01 2.5525 2.4018e+00 2.8229e-01 2.8229e-01
 Queue2 ClosedClass 2.2300e-01 1.0758e-01 0.5183 2.9735e-02 4.3030e-01 4.3030e-01
 Queue2   OpenClass 2.0707e-01 9.9722e-02 0.7342 6.9025e-01 2.8206e-01 2.8206e-01
 Queue3 ClosedClass 1.4357e-01 4.7811e-02 0.3337 1.9144e-02 4.3030e-01 4.3030e-01
 Queue3   OpenClass 1.6297e-01 5.4272e-02 0.5779 5.4324e-01 2.8201e-01 2.8201e-01
 Queue4 ClosedClass 1.0758e-01 2.6894e-02 0.2500 1.4344e-02 4.3030e-01 4.3030e-01
 Queue4   OpenClass 9.4332e-16 2.3581e-16 0.5000 3.1444e-15 1.8865e-15 1.8865e-15
 Queue5 ClosedClass 1.5000e+00 2.9999e-01 0.2000 2.0001e-01 7.4998e+00 7.4998e+00
 Queue5   OpenClass 1.2614e-01 2.5219e-02 0.4474 4.2046e-01 2.8196e-01 2.8196e-01
 Source ClosedClass 0.0000e+00 0.0000e+00 0.0000 0.0000e+00 1.0000e+00 1.0000e+00
 Source   OpenCl

Station    JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
 Queue1 ClosedClass 2.2455 0.6707 3.3298  3.3298 0.6750 0.6750
 Queue1   OpenClass 1.4007 0.2991 4.5988  4.5988 0.3015 0.3015
 Queue2 ClosedClass 0.3641 0.1705 0.5387  0.5387 0.6750 0.6750
 Queue2   OpenClass 0.2290 0.1082 0.7567  0.7567 0.3015 0.3015
 Queue3 ClosedClass 0.2249 0.0750 0.3349  0.3349 0.6750 0.6750
 Queue3   OpenClass 0.1764 0.0586 0.5752  0.5752 0.3015 0.3015
 Queue4 ClosedClass 0.1679 0.0420 0.2487  0.2487 0.6750 0.6750
 Queue4   OpenClass 0.0000 0.0000 0.0000  0.0000 0.0000 0.0000
 Queue5 ClosedClass 0.0000 0.0000 0.0000  0.0000 0.0000 0.0000
 Queue5   OpenClass 0.1358 0.0272 0.4457  0.4457 0.3015 0.3015
 Source ClosedClass 0.0000 0.0000 0.0000  0.0000 0.0000 1.0000
 Source   OpenClass 0.0000 0.0000 0.0000  0.0000 0.0000 0.3000
   Station     JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Queue1  ClosedClass  2.2455  0.6707  3.3298  3.3298  0.6750  0.6750
1   Queue1    OpenClass  1.4007  0.29

Station    JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
 Queue1 ClosedClass 2.2980 0.6510 3.6515  3.6515 0.6293 0.6293
 Queue1   OpenClass 1.5507 0.3265 4.6409  4.6409 0.3341 0.3341
 Queue2 ClosedClass 0.3347 0.1591 0.5319  0.5319 0.6293 0.6293
 Queue2   OpenClass 0.2382 0.1120 0.7129  0.7129 0.3341 0.3341
 Queue3 ClosedClass 0.2070 0.0689 0.3291  0.3291 0.6289 0.6289
 Queue3   OpenClass 0.1914 0.0636 0.5730  0.5730 0.3341 0.3341
 Queue4 ClosedClass 0.1603 0.0401 0.2549  0.2549 0.6289 0.6289
 Queue5   OpenClass 0.1485 0.0297 0.4446  0.4446 0.3341 0.3341
 Source ClosedClass 0.0000 0.0000 0.0000  0.0000 0.0000 1.0000
 Source   OpenClass 0.0000 0.0000 0.0000  0.0000 0.0000 0.3000
  Station     JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  Queue1  ClosedClass  2.2980  0.6510  3.6515  3.6515  0.6293  0.6293
1  Queue1    OpenClass  1.5507  0.3265  4.6409  4.6409  0.3341  0.3341
2  Queue2  ClosedClass  0.3347  0.1591  0.5319  0.5319  0.6293  0.6293
3  Queue2    OpenClass 

/home/gcasale/Dropbox/code/line-dev.git/python/line_solver/api/solvers/mva/amvald.py:271: RuntimeWarning: invalid value encountered in divide
  deltaclass = (Nchain - 1) / Nchain
/home/gcasale/Dropbox/code/line-dev.git/python/line_solver/api/solvers/mva/amvald.py:119: RuntimeWarning: invalid value encountered in divide
  deltaclass = (Nchain_in - 1) / Nchain_in
/home/gcasale/Dropbox/code/line-dev.git/python/line_solver/api/solvers/mva/amvald.py:226: RuntimeWarning: overflow encountered in scalar multiply
  Wchain[k, r] = STeff[k, r] * (1 + selfArvlQlenSeenByClosed[k, r])
/home/gcasale/Dropbox/code/line-dev.git/python/line_solver/api/solvers/mva/amvald.py:138: RuntimeWarning: invalid value encountered in scalar subtract
  totArvlQlenSeenByClosed[k, r] = (deltaclass[r] * Qchain_in[k, r] +
/home/gcasale/Dropbox/code/line-dev.git/python/line_solver/api/solvers/mva/amvald.py:337: RuntimeWarning: invalid value encountered in subtract
  while (inner_iter < 2 or np.max(np.abs(Qchain - Qchain_1

Station    JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
 Queue1 ClosedClass 2.1961 0.6901 3.2598  3.2598 0.6842 0.6841
 Queue1   OpenClass 1.2323 0.2811 4.4293  4.4293 0.2920 0.2918
 Queue2 ClosedClass 0.3860 0.1721 0.5397  0.5397 0.6841 0.6841
 Queue2   OpenClass 0.2253 0.1035 0.7567  0.7567 0.2918 0.2918
 Queue3 ClosedClass 0.2384 0.0760 0.3345  0.3345 0.6841 0.6841
 Queue3   OpenClass 0.1716 0.0560 0.5764  0.5764 0.2918 0.2918
 Queue4 ClosedClass 0.1795 0.0430 0.2512  0.2512 0.6841 0.6841
 Queue5   OpenClass 0.1305 0.0256 0.4379  0.4379 0.2918 0.2918
 Source ClosedClass 0.0000 0.0000 0.0000  0.0000 0.0000 1.0000
 Source   OpenClass 0.0000 0.0000 0.0000  0.0000 0.0000 0.3000
  Station     JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  Queue1  ClosedClass  2.1961  0.6901  3.2598  3.2598  0.6842  0.6841
1  Queue1    OpenClass  1.2323  0.2811  4.4293  4.4293  0.2920  0.2918
2  Queue2  ClosedClass  0.3860  0.1721  0.5397  0.5397  0.6841  0.6841
3  Queue2    OpenClass 